# Linear Optimization
Both Objective Function and Constraints are linear

In [1]:
import sys
import docplex.mp
import random

# first import the Model class from docplex.mp
from docplex.mp.model import Model

In [ ]:
# create one model instance, with a name
m = Model(name='AMC Linear Optimization')

# by default, all variables in Docplex have a lower bound of 0 and infinite upper bound
x1 = m.continuous_var(name='x1')
x2 = m.continuous_var(name='x2')
x3 = m.continuous_var(name='x3')
# write constraints
m.add_constraint(x1 + 2 * x2 - x3 <= 24.0)
m.add_constraint(x1 + x2 + x3 >= 15.0)
m.add_constraint(x2 - x3 <= 5.0)

m.maximize(2 * x1 + 3 * x2 - 6*x3)

In [70]:
def getLinearExpression(model, variableList, coeffList):
    from docplex.mp.linear import LinearExpr
    lnExpr = LinearExpr(model)
    if len(coeffList)>0:
        count=0
        for var in variableList:
            lnExpr.add(coeffList[count]*var)
            count=count+1
    else:
        for var in variableList:
            lnExpr.add(var)
        
    return (lnExpr)

def get_generic_MILP(num_binaryVars, num_continuousVars):
    # construct model using docplex
    m = Model('AMC Linear Optimization for 3ADMM-H')

    if num_binaryVars < 1 or num_continuousVars < 1:
        print('Both the Binary Variables and Continuous Variables should be at least 1')
        print('Using 1 binary variable and 1 continuous variable')
        num_binaryVars=1
        num_continuousVars=1
        
    constraints = 1
    binary_size = num_binaryVars
    size = num_continuousVars
    if (int)((num_binaryVars+num_continuousVars)/3) > 1:
        constraints = random.randint(1, (int)((2*num_binaryVars+num_continuousVars)/2.5))
    x_vals = [random.randint(0,1) for i in range(binary_size)]
    y_vals = [random.randint(0,1) for i in range(binary_size)]
    u_vals = [random.randint(0,20) for i in range(size)]
    xs = m.binary_var_list(binary_size, name='x')
    ys = m.binary_var_list(binary_size, name='y')
    us = m.continuous_var_list(size, name='u')

    # Binary and Continuous variable must be separable in the objective
    obj = m.sum([xs[i] for i in range(binary_size)]) + m.sum([ys[i] for i in range(binary_size)]) + m.sum([us[i] for i in range(size)])
    m.minimize(obj)
    for i in range(constraints):
        xList = random.choices(range(binary_size), k = 3)  
        yList = random.choices(range(binary_size), k = 2)  
        uList = random.choices(range(size), k = 2)
        variableList=[]
        coeffList=[]
        value=0
        for i in xList:
            variableList.append(xs[i])
            coeff=random.randint(1,90)
            coeffList.append(coeff)
            value = value + coeff*x_vals[i]
        for i in yList:
            variableList.append(ys[i])
            coeff=random.randint(1,90)
            coeffList.append(coeff)
            value = value + coeff*y_vals[i]
        for i in uList:
            variableList.append(us[i])
            coeff=random.randint(1,90)
            coeffList.append(coeff)
            value = value + coeff*u_vals[i]

        m.add_constraint(getLinearExpression(m,variableList,coeffList) == value, 'Constraint')
    return(m)


In [71]:
binary_vars=0
int_vars=0
continuous_vars=0
m = get_generic_MILP(binary_vars,continuous_vars)

Both the Binary Variables and Continuous Variables should be at least 1
Using 1 binary variable and 1 continuous variable


In [72]:
m.print_information()

Model: AMC Linear Optimization for 3ADMM-H
 - number of variables: 3
   - binary=2, integer=0, continuous=1
 - number of constraints: 1
   - linear=1
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [73]:
print(m.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Linear Optimization for 3ADMM-H

Minimize
 obj: x_0 + y_0 + u_0
Subject To
 Constraint: 83 x_0 + 58 y_0 + 32 u_0 = 576

Bounds
 0 <= x_0 <= 1
 0 <= y_0 <= 1

Binaries
 x_0 y_0
End



In [75]:
from docplex.mp.constants import int_probtype_to_string
filename='..\\problemFiles\\001-Linear Optimization Problem-'
varText=''
if m.number_of_binary_variables>0:
    varText='B%03d'%(m.number_of_binary_variables)+'-'
if m.number_of_integer_variables>0:    
    varText=varText+'I%03d'%(m.number_of_integer_variables)+'-'
if m.number_of_continuous_variables>0:    
    varText=varText+'C%03d'%(m.number_of_continuous_variables)+'-'
cpx = m.get_cplex(do_raise=False)
if cpx:
    cpx_probtype = cpx.get_problem_type()
    filename=filename+int_probtype_to_string(cpx_probtype)+'-'

filename=filename+varText+str(random.randint(122100,999999))
m.export_as_lp(filename)

'..\\problemFiles\\001-Linear Optimization Problem-MILP-B002-C001-348085.lp'